In [1]:
# =========================
# IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_plots import *

from GGS_main import *

# =========================

c:\Users\sal_f\anaconda3\envs\env_ggs0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

### GGS Executable

In [2]:
power=1
path="local"
config_name="test"

In [3]:
if config_name is None:
    print("No config file specified. Exiting.")
    exit(1)

config = GGS_config_import(config_name)

target_datetime = config['MISSION'].get('target_date')
if not target_datetime:
    print("Issue with target datetime. Using current datetime.")
    target_datetime = dt.datetime.now(dt.timezone.utc)

if path == "local":
    root_directory = GGS_config_process(config, path="default")
elif path == "rucool":
    root_directory = GGS_config_process(config, path="/www/web/rucool/hurricane/model_comparisons/maps/yucatan")
else:
    raise ValueError("Invalid root directory.")

datetime_list = [target_datetime.replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%dT%H:%M:%SZ')]
datetime_index = datetime_list[0]

glider_dataframes = None
if config['PLOT'].get('show_gliders'):
    min_lat, min_lon = config['MISSION']['extent'][0]
    max_lat, max_lon = config['MISSION']['extent'][1]
    search_extent = [min_lon, max_lon, min_lat, max_lat]
    glider_dataframes = acquire_gliders(
        extent=search_extent,
        target_date=target_datetime,
        date_delta=dt.timedelta(days=1),
        requested_variables=["time", "longitude", "latitude", "profile_id", "depth"],
        print_vars=False,
        target="all",
        request_timeout=5,
        enable_parallel=False
    )


### IMPORTING GGS CONFIGURATION: test ###

Configuration import success!

### PROCESSING GGS CONFIGURATION ###

Configuration:

--> MISSION
mission_name: TEST
target_date: 2024-04-02T22:33:41+0000
max_depth: 1000
extent: ((15, -90), (30, -78))
GPS_coords: None
glider_id: None

--> MODEL
single_datetime: True
enable_rtofs: False
enable_cmems: True
enable_gofs: False
save_model_data: False
save_depth_average: True
save_bin_average: False
chunk: True

--> PLOT
latitude_qc: 21.175
longitude_qc: -86.185
density: 3
mag1: 0.0
mag2: 0.2
mag3: 0.3
mag4: 0.4
mag5: 0.5
tolerance: 7.5
show_gliders: False
show_route: False
show_eez: True
show_qc: False
manual_extent: None

--> DATA
bathymetry_path: c:\Users\sal_f\OneDrive\Desktop\STF-0\!-GGS\GGS_Scripts\data/bathymetry/GEBCO_2023_sub_ice_topo.nc
eez_path: c:\Users\sal_f\OneDrive\Desktop\STF-0\!-GGS\GGS_Scripts\data/eez/eez_boundaries_v12.shp
reprocess: False



In [4]:
datetime_index = datetime_index
config_flag = config
root_directory_flag = root_directory
glider_data_flag = glider_dataframes

enable_rtofs_flag = config_flag['MODEL']['enable_rtofs']
enable_cmems_flag = config_flag['MODEL']['enable_cmems']
enable_gofs_flag = config_flag['MODEL']['enable_gofs']
save_model_data_flag = config_flag['MODEL']['save_model_data']
save_depth_average_flag = config_flag['MODEL']['save_depth_average']
save_bin_average_flag = config_flag['MODEL']['save_bin_average']
chunk_flag = config_flag['MODEL']['chunk']

latitude_qc_flag = config_flag['PLOT']['latitude_qc']
longitude_qc_flag = config_flag['PLOT']['longitude_qc']
density_flag = config_flag['PLOT']['density']
mag1_flag = config_flag['PLOT']['mag1']
mag2_flag = config_flag['PLOT']['mag2']
mag3_flag = config_flag['PLOT']['mag3']
mag4_flag = config_flag['PLOT']['mag4']
mag5_flag = config_flag['PLOT']['mag5']
tolerance_flag = config_flag['PLOT']['tolerance']
show_route_flag = config_flag['PLOT']['show_route']
show_eez_flag = config_flag['PLOT']['show_eez']
show_qc_flag = config_flag['PLOT']['show_qc']
manual_extent_flag = config_flag['PLOT']['manual_extent']

sub_directory_plots = os.path.join(root_directory_flag, "plots", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_plots, exist_ok=True)
sub_directory_data = os.path.join(root_directory_flag, "data", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_data, exist_ok=True)

check_datetime = pd.to_datetime(datetime_index).strftime('%Y%m%dT%HZ')
check_pattern = os.path.join(sub_directory_data, f"*_DepthAverageData_{check_datetime}.nc")
check_files = glob.glob(check_pattern)
if check_files:
    print(f"Datetime {datetime_index} already processed: {check_files[0]}, skipping task.")
else:
    print(f"Datetime {datetime_index} unprocessed, proceeding with task.")

Datetime 2024-04-02T00:00:00Z unprocessed, proceeding with task.


In [5]:
model_datasets = []
    
if enable_rtofs_flag:
    try:
        rtofs = RTOFS()
        rtofs.rtofs_load(config_flag, datetime_index)
        rtofs.rtofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
        rtofs_model_data = rtofs.data
        
        rtofs_depth_average, rtofs_bin_average = interpolate_rtofs(config_flag, sub_directory_data, rtofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)

        rtofs_datasets = (rtofs_model_data, rtofs_depth_average, rtofs_bin_average)
        model_datasets.append(rtofs_datasets)
    except Exception as e:
        print(f"Error during RTOFS processing: {e}")

if enable_cmems_flag:
    try:
        cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
        cmems.cmems_load(config_flag, datetime_index)
        cmems.cmems_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
        cmems_model_data = cmems.data
        
        cmems_depth_average, cmems_bin_average = interpolate_cmems(config_flag, sub_directory_data, cmems_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
        
        cmems_datasets = (cmems_model_data, cmems_depth_average, cmems_bin_average)
        model_datasets.append(cmems_datasets)
    except Exception as e:
        print(f"Error during CMEMS processing: {e}")

if enable_gofs_flag:
    try:
        gofs = GOFS()
        gofs.gofs_load(config_flag, datetime_index)
        gofs.gofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
        gofs_model_data = gofs.data

        gofs_depth_average, gofs_bin_average = interpolate_gofs(config_flag, sub_directory_data, gofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)

        gofs_datasets = (gofs_model_data, gofs_depth_average, gofs_bin_average)
        model_datasets.append(gofs_datasets)
    except Exception as e:
        print(f"Error during GOFS processing: {e}")


### MODEL DATA: [CMEMS] ###

INFO - 2024-04-02T22:33:42Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-04-02T22:33:42Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-02T22:33:44Z - Service was not specified, the default one was selected: "arco-geo-series"

### INTERPOLATING CMEMS MODEL DATA ###

Start time (UTC): 2024-04-02 22:33:47.894629
End time (UTC): 2024-04-02 22:34:06.372952
Run time: 0:00:18.478323


In [ ]:
GGS_plot_magnitude(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    density=density_flag,
    gliders=glider_data_flag,
    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
    manual_extent=manual_extent_flag
)
GGS_plot_threshold(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    density=density_flag,
    mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
    gliders=glider_data_flag,
    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
    manual_extent=manual_extent_flag
)
GGS_plot_advantage(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    density=density_flag,
    tolerance=tolerance_flag,
    mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
    gliders=glider_data_flag,
    show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
    manual_extent=manual_extent_flag
)
GGS_plot_profiles(
    config_flag,
    sub_directory_plots,
    datetime_index,
    model_datasets,
    latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
    threshold=0.5
)

---

### GGS Testing

In [6]:
cmems_depth_average = model_datasets[0][1]
cmems_depth_average

<xarray.Dataset>
Dimensions:        (time: 1, lat: 181, lon: 145)
Coordinates:
  * lat            (lat) float32 15.0 15.08 15.17 15.25 ... 29.83 29.92 30.0
  * lon            (lon) float32 -90.0 -89.92 -89.83 ... -78.17 -78.08 -78.0
Dimensions without coordinates: time
Data variables:
    u_depth_avg    (time, lat, lon) float64 nan nan nan ... 0.03152 0.02758
    v_depth_avg    (time, lat, lon) float64 nan nan nan ... -0.2567 -0.2081
    mag_depth_avg  (time, lat, lon) float64 nan nan nan ... 0.2794 0.269 0.235
    dir_depth_avg  (time, lat, lon) float64 nan nan nan ... 270.3 269.9 271.1
Attributes:
    model_datetime:  2024-04-02T00:00:00
    model_name:      CMEMS

In [7]:
import rasterio
from rasterio.transform import from_bounds
import matplotlib.pyplot as plt
import numpy as np

def save_georeferenced_image(plot_array, extent, output_path):
    
    if plot_array.ndim == 2:
        plot_array = np.expand_dims(plot_array, axis=2)
    elif plot_array.shape[2] > 3:
        plot_array = plot_array[:, :, :3]
    
    transform = from_bounds(*extent, width=plot_array.shape[1], height=plot_array.shape[0])
    
    metadata = {
        'driver': 'GTiff',
        'height': plot_array.shape[0],
        'width': plot_array.shape[1],
        'count': plot_array.shape[2],
        'dtype': 'uint8',
        'crs': '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
        'transform': transform
    }
    
    with rasterio.open(output_path, 'w', **metadata) as dst:
        for i in range(plot_array.shape[2]):
            dst.write(plot_array[:, :, i], i + 1)

In [20]:
### FUNCTION:
def GGS_threshold_geotiff(config, directory, datetime_index, model_datasets, latitude_qc=None, longitude_qc=None, density=2, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, gliders=None, show_route=False, show_eez=False, show_qc=False, manual_extent=None):
    
    '''
    Create a georeferenced TIFF image of the threshold plot for the Glider Guidance System.
    
    Args:
    - config (dict): Glider Guidance System mission configuration.
    - directory (str): Directory to save the plot.
    - datetime_index (int): Index of the datetime for the plot title.
    - model_datasets (tuple): Tuple containing the model datasets.
    - latitude_qc (float): Latitude for QC plotting.
    - longitude_qc (float): Longitude for QC plotting.
    - density (int): Density of the streamplot.
    - mag1 (float): Threshold for the first magnitude level.
    - mag2 (float): Threshold for the second magnitude level.
    - mag3 (float): Threshold for the third magnitude level.
    - mag4 (float): Threshold for the fourth magnitude level.
    - mag5 (float): Threshold for the fifth magnitude level.
    - gliders (optional): DataFrame containing glider data for plotting.
    - show_route (bool): Flag to show the glider route.
    - show_qc (bool): Flag to show the QC sample point.
    - show_eez (bool): Flag to show the Exclusive Economic Zone (EEZ).
    - manual_extent (list or None): Manual specification of plot extent.

    Returns:
    - None
    '''

    print(f"\n### CREATING THRESHOLD PLOT ###\n")
    start_time = print_starttime()

    valid_datasets = [datasets for datasets in model_datasets if datasets is not None]
    num_datasets = len(valid_datasets)
    if num_datasets == 0:
        print("No datasets provided for plotting.")
        end_time = print_endtime()
        print_runtime(start_time, end_time)
        return

    def plot_threshold(ax, config, model_depth_average, latitude_qc, longitude_qc, density, mag1, mag2, mag3, mag4, mag5, gliders, show_route, show_qc, show_eez, manual_extent):
        
        longitude = model_depth_average.lon.values.squeeze()
        latitude = model_depth_average.lat.values.squeeze()
        u_depth_avg = model_depth_average['u_depth_avg'].values.squeeze()
        v_depth_avg = model_depth_average['v_depth_avg'].values.squeeze()
        mag_depth_avg = model_depth_average['mag_depth_avg'].values.squeeze()
        
        if manual_extent is not None and len(manual_extent) == 2 and all(len(sublist) == 2 for sublist in manual_extent):
            map_extent = [manual_extent[0][1], manual_extent[1][1], manual_extent[0][0], manual_extent[1][0]]
        else:
            data_extent_lon = [float(longitude.min()), float(longitude.max())]
            data_extent_lat = [float(latitude.min()), float(latitude.max())]
            map_extent = data_extent_lon + data_extent_lat
        ax.set_extent(map_extent, crs=ccrs.PlateCarree())

        plot_threshold_zones(ax, longitude, latitude, mag_depth_avg, mag1, mag2, mag3, mag4, mag5, threshold_legend=True)
        plot_streamlines(ax, longitude, latitude, u_depth_avg, v_depth_avg, density=density)

        if gliders is not None:
            plot_add_gliders(ax, gliders, legend=True)
            glider_legend = ax.get_legend()
            if glider_legend:
                glider_legend.get_frame().set_alpha(0.5)
                glider_legend.get_frame().set_facecolor('white')
                ax.add_artist(glider_legend)

        if show_route:
            plot_glider_route(ax, config)
        
        if show_qc:
            (y_index, x_index), (lat_index, lon_index) = calculate_gridpoint(model_depth_average, latitude_qc, longitude_qc)
            qc_lon = model_depth_average['lon'].isel(x=x_index, y=y_index).values
            qc_lat = model_depth_average['lat'].isel(x=x_index, y=y_index).values
            circle = Circle((qc_lon, qc_lat), radius=0.25, edgecolor='purple', facecolor='none', linewidth=2, transform=ccrs.PlateCarree(), zorder=95)
            ax.add_patch(circle)

        if show_eez:
            plot_add_eez(ax, config, color='dimgrey', linewidth=3, zorder=90)
        
        try:
            plot_bathymetry(ax, config, model_depth_average, isobath1=-100, isobath2=-1000, downsample="auto", show_legend=False)
            bathymetry_legend = ax.get_legend()
            if bathymetry_legend:
                bathymetry_legend.get_frame().set_alpha(0.5)
                ax.add_artist(bathymetry_legend)
        except:
            print(f"WARNING: Bathymetry contouring was unsuccessful for {model_depth_average.attrs['model_name']}. Using default ocean color instead.")
            ax.add_feature(cfeature.OCEAN, zorder=1)

        ax.add_feature(cfeature.GSHHSFeature(scale='full'), edgecolor="black", facecolor="tan", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.RIVERS, edgecolor="steelblue", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.LAKES, edgecolor="black", facecolor="lightsteelblue", linewidth=0.25, zorder=90)
        ax.add_feature(cfeature.BORDERS, edgecolor="black", linewidth=0.25, zorder=90)
        
        model_name = model_depth_average.attrs['model_name']
        fig.canvas.draw()
        ax.set_axis_off()
        plt.tight_layout(pad=0)
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))        
        def save_georeferenced_image(image, map_extent, output_filename):
            height, width, _ = image.shape
            transform = from_bounds(map_extent[0], map_extent[2], map_extent[1], map_extent[3], width, height)
            
            with rasterio.open(
                output_filename, 'w', driver='GTiff', height=height, width=width,
                count=3, dtype=image.dtype, crs='+proj=latlong',
                transform=transform) as dst:
                dst.write(image[:, :, 0], 1)  # Red channel
                dst.write(image[:, :, 1], 2)  # Green channel
                dst.write(image[:, :, 2], 3)  # Blue channel

        save_georeferenced_image(image, map_extent, f'{model_name}_georeferenced_plot.tif')

    fig, axs = plt.subplots(1, num_datasets, subplot_kw={'projection': ccrs.Mercator()}, figsize=(10*num_datasets, 10))
    if num_datasets == 1:
        axs = [axs]

    for ax, (model_data, depth_average_data, bin_average_data) in zip(axs, valid_datasets):
        plot_threshold(ax, config, depth_average_data, latitude_qc, longitude_qc, density, mag1, mag2, mag3, mag4, mag5, gliders, show_route, show_qc, show_eez, manual_extent)
    
    plt.close(fig)

    end_time = print_endtime()
    print_runtime(start_time, end_time)

In [21]:
GGS_plot_threshold(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
        gliders=glider_data_flag,
        show_route=show_route_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag
    )


### CREATING THRESHOLD PLOT ###

Start time (UTC): 2024-04-02 23:08:40.941556


C:\Users\sal_f\AppData\Local\Temp\ipykernel_6324\1533927385.py:100: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)


End time (UTC): 2024-04-02 23:09:41.945975
Run time: 0:01:01.004419


---